## Import Libraries

In [2]:
import pandas as pd
import numpy as np
from glob import glob
import os
import json
from PIL import Image
import albumentations as A
import random
import os
from os.path import exists
#import glob
import json
import shutil

In [8]:
train_path = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/amy-def-mfg-jsons"
val_path = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/amy-def-mfg-test-jsons"

train_df = pd.DataFrame({"json_path":glob(os.path.join(train_path,"*.json"))})
val_df = pd.DataFrame({"json_path":glob(os.path.join(val_path,"*.json"))})

## Get count of plaques for train/validation

In [12]:
def get_count(l):
    """Get count of plaque types 

    Args:
        l (json): json file of annotated WSI

    Returns:
        dict type: Return count of plaque against their type
    """
    with open(l) as f:
        data = json.load(f)
    plaque_dict = {'Cored': 0, 'Diffuse': 0, 'Coarse-Grained': 0,'CAA': 0}
    for tileId, ele in data.items():
        for region in ele:
            if "label" in region.keys():
                if "name" in region['label'].keys():
                    plaque_dict[region['label']["name"]]=plaque_dict[region['label']["name"]]+1
    return plaque_dict
                

In [13]:
# Get count of all files under json_path
for cls in ["Cored","Diffuse","Coarse-Grained","CAA"]:
    train_df[cls] = train_df["json_path"].apply(lambda l:get_count(l)[cls] )

In [14]:
# Get count of all files under json_path (validation)
for cls in ["Cored","Diffuse","Coarse-Grained","CAA"]:
    val_df[cls] = val_df["json_path"].apply(lambda l:get_count(l)[cls] )

In [15]:

# format file name
train_df["file_name"] = train_df["json_path"].apply(lambda l: l.split("/")[-1].split(".")[0])
val_df["file_name"] = val_df["json_path"].apply(lambda l: l.split("/")[-1].split(".")[0])

In [17]:
train_df

,json_path,Cored,Diffuse,Coarse-Grained,CAA,file_name
0,/gladstone/finkbeiner/steve/work/data/npsad_da...,21,26,3,0,XE07-047_1_AmyB_1
1,/gladstone/finkbeiner/steve/work/data/npsad_da...,5,34,3,0,XE18-066_1_AmyB_1
2,/gladstone/finkbeiner/steve/work/data/npsad_da...,13,18,4,8,XE07-064_1_AmyB_1
3,/gladstone/finkbeiner/steve/work/data/npsad_da...,2,27,3,9,XE17-004_1_AmyB_1
4,/gladstone/finkbeiner/steve/work/data/npsad_da...,23,22,14,0,XE15-039_1_AmyB_1
5,/gladstone/finkbeiner/steve/work/data/npsad_da...,1,20,0,2,XE19-037_1_AmyB_1
6,/gladstone/finkbeiner/steve/work/data/npsad_da...,10,13,10,9,XE17-039_1_AmyB_1
7,/gladstone/finkbeiner/steve/work/data/npsad_da...,27,97,5,0,XE07-048_1_AmyB_1
8,/gladstone/finkbeiner/steve/work/data/npsad_da...,26,27,6,6,XE15-022_1_AmyB_1
9,/gladstone/finkbeiner/steve/work/data/npsad_da...,6,20,10,6,XE10-045_1_AmyB_1


In [18]:
test_files =  [x.split("/")[-1] for x in glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/test","*"))]

In [22]:
#val_files =  [x.split("/")[-1] for x in glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val","*"))]

In [23]:
val_files =  ['XE12-016_1_AmyB_1',
 'XE13-007_1_AmyB_1',
 'XE14-004_1_AmyB_1',
 'XE16-021_1_AmyB_1',
 'XE17-010_1_AmyB_1',
 'XE17-030_1_AmyB_1']

In [24]:
val_df["flag"] = val_df["file_name"].apply(lambda l: "val" if l in val_files else "train")

In [25]:
train_df["flag"] = train_df["file_name"].apply(lambda l: "test" if l in test_files else "train")

In [26]:
all_data_df = pd.concat([train_df,val_df], ignore_index=True)

In [27]:
all_data_df.groupby(["flag"]).sum()

,Cored,Diffuse,Coarse-Grained,CAA
flag,,,,
test,26,85,22,15
train,260,396,148,70
val,42,73,55,9


In [29]:
all_data_df.groupby(["flag"]).count()

,json_path,Cored,Diffuse,Coarse-Grained,CAA,file_name
flag,,,,,,
test,5,5,5,5,5,5
train,18,18,18,18,18,18
val,5,5,5,5,5,5


In [28]:
all_data_df

,json_path,Cored,Diffuse,Coarse-Grained,CAA,file_name,flag
0,/gladstone/finkbeiner/steve/work/data/npsad_da...,21,26,3,0,XE07-047_1_AmyB_1,train
1,/gladstone/finkbeiner/steve/work/data/npsad_da...,5,34,3,0,XE18-066_1_AmyB_1,test
2,/gladstone/finkbeiner/steve/work/data/npsad_da...,13,18,4,8,XE07-064_1_AmyB_1,train
3,/gladstone/finkbeiner/steve/work/data/npsad_da...,2,27,3,9,XE17-004_1_AmyB_1,train
4,/gladstone/finkbeiner/steve/work/data/npsad_da...,23,22,14,0,XE15-039_1_AmyB_1,train
5,/gladstone/finkbeiner/steve/work/data/npsad_da...,1,20,0,2,XE19-037_1_AmyB_1,test
6,/gladstone/finkbeiner/steve/work/data/npsad_da...,10,13,10,9,XE17-039_1_AmyB_1,train
7,/gladstone/finkbeiner/steve/work/data/npsad_da...,27,97,5,0,XE07-048_1_AmyB_1,train
8,/gladstone/finkbeiner/steve/work/data/npsad_da...,26,27,6,6,XE15-022_1_AmyB_1,train
9,/gladstone/finkbeiner/steve/work/data/npsad_da...,6,20,10,6,XE10-045_1_AmyB_1,train


## Split Train/Test Data

In [35]:
train_files = list(df1[df1["flag"]=="train"]["file_name"].values)
train_files.remove("XE19-010_1_AmyB_1")

In [37]:
DATASET_PATH = "/home/mahirwar/Desktop/Monika/npsad_data/vivek/Datasets/amyb_wsi_v2"
TARGET_PATH = '/home/mahirwar/Desktop/Monika/npsad_data/vivek/Datasets/amyb_wsi_v2/train_2' # use either train or train_v2
target_folders = ["images","labels"]
origin_folders = ["image","mask"]
for i in train_files:
   for target_folder,origin_folder in zip(target_folders,origin_folders):
      if i==".DS_Store" :
         continue
      origin = os.path.join(DATASET_PATH,i,origin_folder)
      target = os.path.join(TARGET_PATH,target_folder)
      if not os.path.exists(target):
         os.makedirs(target)
      #print(origin)
      # Fetching the list of all the files
      files = os.listdir(origin)
      #print(files)
      # Fetching all the files to directory
      for file_name in files:
         shutil.copyfile(os.path.join(origin,file_name), os.path.join(target,file_name))
      print("Files are copied successfully")

Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are 

In [56]:
DATASET_PATH = "/home/mahirwar/Desktop/Monika/npsad_data/vivek/Datasets/amyb_wsi_v2"
TARGET_PATH = '/home/mahirwar/Desktop/Monika/npsad_data/vivek/Datasets/amyb_wsi_v2/val_2' # use either train or train_v2
target_folders = ["images","labels"]
origin_folders = ["image","mask"]
for i in val_files:
   for target_folder,origin_folder in zip(target_folders,origin_folders):
      if i==".DS_Store" :
         continue
      origin = os.path.join(DATASET_PATH,i,origin_folder)
      target = os.path.join(TARGET_PATH,target_folder)
      if not os.path.exists(target):
         os.makedirs(target)
      #print(origin)
      # Fetching the list of all the files
      files = os.listdir(origin)
      #print(files)
      # Fetching all the files to directory
      for file_name in files:
         shutil.copyfile(os.path.join(origin,file_name), os.path.join(target,file_name))
      print("Files are copied successfully")

Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully
Files are copied successfully


## Data Augmentation

In [38]:
# Check count of labels on folders
all_images = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/train_2/images/",'*.png'))
all_masks = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/train_2/labels/",'*.png'))

all_images.sort()
all_masks.sort()

cored_images, cored_masks, diffuse_images, diffuse_masks, cg_images,cg_masks, caa_images, caa_masks  = [],[],[],[],[],[],[],[]
for image, mask in zip(all_images,all_masks):
    found = np.unique(np.array(Image.open(mask)))
    if 50 in found:
        cored_images.append(image)
        cored_masks.append(mask)
    if 100 in found:
        diffuse_images.append(image)
        diffuse_masks.append(mask)
    if 150 in found:
        cg_images.append(image)
        cg_masks.append(mask)
    if 200 in found:
        caa_images.append(image)
        caa_masks.append(mask)

In [39]:
len(all_images)

529

In [40]:
len(cored_images),  len(diffuse_images), len(cg_images), len(caa_images)

(201, 251, 135, 82)

In [42]:
# Use transformations for image
transforms  = A.Compose([ A.VerticalFlip(p=0.5),
                            A.HorizontalFlip(p=0.5),
                            A.Blur(blur_limit=1,p=0.2),
                            A.RandomRotate90(),
                        ])

# select files randomly
def get_randimages_dataug(total_imgs, image_filenames, label_filenames):
    image_filenames.sort()
    label_filenames.sort()
    random_image_file = []
    random_label_file = []
    for i in range(total_imgs):
        random.seed(i)
        random_image_file.append(random.choice(image_filenames))
        random.seed(i)
        random_label_file.append(random.choice(label_filenames))
    return [random_image_file, random_label_file]

# upsample dataset with the augmentations
def upsample_dataset(dataset_base_dir, random_img_filenames, rand_label_filenames, variations, transforms, dest_img_folder_name, dest_label_folder_name):
    i = 0
    aug_img_files = []
    aug_mask_files = []

    # Make dir where tha augmented file will reside
    aug_img_dir = os.path.join(dataset_base_dir, dest_img_folder_name)
    if not os.path.exists(aug_img_dir):
            os.makedirs(aug_img_dir)
            print("Augmented Directory '%s' created" %aug_img_dir)
    
    aug_mask_dir = os.path.join(dataset_base_dir, dest_label_folder_name)
    if not os.path.exists(aug_mask_dir):
            os.makedirs(aug_mask_dir)
            print("Augmented Directory '%s' created" %aug_mask_dir)

    print(os.listdir(dataset_base_dir))
    print("\nData Augmentation in Progress ...")
    total_imgs = len(random_img_filenames)
    
    for i in range(total_imgs):
        # load the image
        img = Image.open(random_img_filenames[i]).convert("RGB")
        img = np.array(img)
        mask = Image.open(rand_label_filenames[i]).convert('P')
        mask = np.array(mask)

        for j in range(variations):
                transformed = transforms(image=img, mask=mask)
                transformed_img = transformed["image"]
                transformed_img = Image.fromarray(transformed_img)

                #To rename the file with prefix A_
                filename = os.path.basename(random_img_filenames[i])
                filepath = os.path.dirname(random_img_filenames[i])

                aug_file_name = "A_" + str(i) + "_" + str(j) + "_" + filename
                new_file = os.path.join(dataset_base_dir,dest_img_folder_name,
                                        aug_file_name)
                transformed_img.save(new_file)
                aug_img_files.append(new_file)

                transformed_mask = transformed["mask"]
                transformed_mask = Image.fromarray(transformed_mask)

                #To rename the file with prefix A_
                filename = os.path.basename(rand_label_filenames[i])
                filepath = os.path.dirname(rand_label_filenames[i])
                aug_file_name = "A_" + str(i) + "_" + str(j) + "_" + filename
                new_file = os.path.join(dataset_base_dir, dest_label_folder_name,
                                        aug_file_name)
                transformed_mask.save(new_file)
                aug_mask_files.append(new_file)
    return aug_img_files, aug_mask_files

In [43]:
aug_value = 70
rand_image_filenames, rand_label_filenames = get_randimages_dataug(aug_value, cg_images, cg_masks)
dataset_base_dir = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/train_2"
aug_img_files, aug_mask_files = upsample_dataset(dataset_base_dir, rand_image_filenames, rand_label_filenames, 1, transforms, "images", "labels")

['labels', 'images']

Data Augmentation in Progress ...


In [44]:
aug_value = 150
rand_image_filenames, rand_label_filenames = get_randimages_dataug(aug_value, caa_images, caa_masks)
dataset_base_dir = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/train_2"
aug_img_files, aug_mask_files = upsample_dataset(dataset_base_dir, rand_image_filenames, rand_label_filenames, 1, transforms, "images", "labels")

['labels', 'images']

Data Augmentation in Progress ...


In [ ]:
aug_value = 150
rand_image_filenames, rand_label_filenames = get_randimages_dataug(aug_value, caa_images, caa_masks)
dataset_base_dir = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/train_2"
aug_img_files, aug_mask_files = upsample_dataset(dataset_base_dir, rand_image_filenames, rand_label_filenames, 1, transforms, "images", "labels")

In [47]:
# Check the count again
all_images = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/train_2/images/",'*.png'))
all_masks = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/train_2/labels/",'*.png'))

all_images.sort()
all_masks.sort()

cored_images, cored_masks, diffuse_images, diffuse_masks, cg_images,cg_masks, caa_images, caa_masks  = [],[],[],[],[],[],[],[]
for image, mask in zip(all_images,all_masks):
    found = np.unique(np.array(Image.open(mask)))
    if 50 in found:
        cored_images.append(image)
        cored_masks.append(mask)
    if 100 in found:
        diffuse_images.append(image)
        diffuse_masks.append(mask)
    if 150 in found:
        cg_images.append(image)
        cg_masks.append(mask)
    if 200 in found:
        caa_images.append(image)
        caa_masks.append(mask)

In [48]:
len(cored_images),  len(diffuse_images), len(cg_images), len(caa_images)

(209, 322, 208, 233)

In [51]:
all_images = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/train_2/images/",'*.png'))
all_masks = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/train_2/labels/",'*.png'))

In [52]:
all_images.sort()
all_masks.sort()

In [53]:
for image, mask in zip(all_images,all_masks):
    found = np.unique(np.array(Image.open(mask)))
    if len(found)<=1:
        print(image)
        os.remove(image)
        os.remove(mask)
        
    

/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/train_2/images/XE10-045_1_AmyB_1_71703x_53045y_image.png
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/train_2/images/XE12-010_1_AmyB_1_30926x_157422y_image.png
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/train_2/images/XE18-001_1_AmyB_1_41989x_106305y_image.png


In [54]:
all_masks_img = [x.split("/")[-1] for x in all_masks]

In [55]:
counter = 0
for image in all_images:
    #print(image.split("/")[-1].replace("image","mask"))
    if image.split("/")[-1].replace("image","mask") not in all_masks_img:
        counter=counter+1
        print(image)
        os.remove(image)
print(counter)

0


In [58]:
all_images = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/val_2/images/",'*.png'))
all_masks = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/val_2/labels/",'*.png'))

In [59]:
all_images.sort()
all_masks.sort()
for image, mask in zip(all_images,all_masks):
    found = np.unique(np.array(Image.open(mask)))
    if len(found)<=1:
        print(image)
        os.remove(image)
        os.remove(mask)

/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/val_2/images/XE13-007_1_AmyB_1_37037x_114458y_image.png
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/val_2/images/XE13-007_1_AmyB_1_88237x_142106y_image.png


In [24]:
all_images = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/interater_data/monika-rater/XE19-037_1_AmyB_1/image",'*.png'))

In [25]:
all_masks = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/interater_data/monika-rater/XE19-037_1_AmyB_1/mask",'*.png'))

In [26]:
all_images.sort()
all_masks.sort()
for image, mask in zip(all_images,all_masks):
    found = np.unique(np.array(Image.open(mask)))
    if len(found)<=1:
        print(image)
        os.remove(image)
        os.remove(mask)

/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/interater_data/monika-rater/XE19-037_1_AmyB_1/image/XE19-037_1_AmyB_1_50176x_173865y_image.png
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/interater_data/monika-rater/XE19-037_1_AmyB_1/image/XE19-037_1_AmyB_1_50176x_175913y_image.png
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/interater_data/monika-rater/XE19-037_1_AmyB_1/image/XE19-037_1_AmyB_1_51200x_176937y_image.png
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/interater_data/monika-rater/XE19-037_1_AmyB_1/image/XE19-037_1_AmyB_1_51200x_177961y_image.png
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/interater_data/monika-rater/XE19-037_1_AmyB_1/image/XE19-037_1_AmyB_1_52224x_175913y_image.png
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/interater_data/monika-rater/XE19-037_1_AmyB_1/image/XE19-037_1_AmyB_1

In [35]:
all_images = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/interrater-study/tiles",'*/image/*.png'))
all_masks = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/interrater-study/tiles",'*/mask/*.png'))

In [36]:
len(all_images)

140

In [37]:
len(all_masks)

140

In [38]:
all_images.sort()
all_masks.sort()
for image, mask in zip(all_images,all_masks):
    if os.path.exists(mask):
        found = np.unique(np.array(Image.open(mask)))
        if len(found)<=1:
    #   if os.path.exists(image):
            os.remove(image)
        #if os.path.exists(mask):
            os.remove(mask)